**source:**
* https://thedatafrog.com/en/articles/docker-influxdb-grafana/

**resources:**
* https://docs.influxdata.com/influxdb/v1.8/introduction/get-started/
* https://www.influxdata.com/blog/getting-started-python-influxdb/
* https://influxdb-python.readthedocs.io/en/latest/

In [2]:
# instaling influxdb library
!pip install influxdb

     |████████████████████████████████| 77 kB 350 kB/s eta 0:00:011
     |████████████████████████████████| 118 kB 1.3 MB/s eta 0:00:01
  Created wheel for msgpack: filename=msgpack-0.6.1-cp38-cp38-linux_x86_64.whl size=343732 sha256=b4ac7b1bffd92aac179f248b95a8c1662987dee3281c432db616bc05893a0d3d
  Stored in directory: /home/jovyan/.cache/pip/wheels/56/2e/45/9ae160fc31c10e4b799d0ebc32ba82b32f1b057e0ebf28ea82
Successfully built msgpack
  Attempting uninstall: msgpack
    Found existing installation: msgpack 1.0.0
    Uninstalling msgpack-1.0.0:
      Successfully uninstalled msgpack-1.0.0


In [3]:
from influxdb import InfluxDBClient
from influxdb import exceptions

In [4]:
try:
    client = InfluxDBClient('localhost', 8086)
    client.ping()
    print('Connected to InfluxDB')
except Exception as e:
    print('Connection error: {}'.format(str(e)))

Connection error: HTTPConnectionPool(host='localhost', port=8086): Max retries exceeded with url: /ping (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f5f923bd370>: Failed to establish a new connection: [Errno 111] Connection refused'))


In [4]:
# Creating a test db

client.create_database('testdb')

In [7]:
# Let's check the list of databases

client.get_list_database()

[{'name': '_internal'}, {'name': 'testdb'}]

To send data to influxdb we have to use a JSON format to indicate 

1.- Measurement (mandatory field)
2.- Tags (optional field)
3.- Time (optional field. If no time value is given Influx db will use the system's timestamp)
4.- Field (mandatory)

In [8]:
# from influx db python documentation
json_body = [
    {
        "measurement": "cpu_load_short",
        "tags": {
            "host": "server01",
            "region": "us-west"
        },
        "time": "2009-11-10T23:00:00Z",
        "fields": {
            "value": 0.64
        }
    }
]


In [10]:
# write_points function also can take de different fields as parameters. See documentation.

client.write_points(json_body, database='testdb')

True

In [17]:
# Let's check the value was stored correctly

result = client.query('select "host", "region", "value" from cpu_load_short;', database='testdb')

In [19]:
# seems like is working just right
print("Result: {0}".format(result))

Result: ResultSet({'('cpu_load_short', None)': [{'time': '2009-11-10T23:00:00Z', 'host': 'server01', 'region': 'us-west', 'value': 0.64}]})
